In [1]:
import pandas as pd
import bigml.api
from pprint import pprint
api = bigml.api.BigML()

In [2]:
training_df = pd.read_csv('./datasets/cs-training.csv')
test_df = pd.read_csv('./datasets/cs-test.csv')

dataframes = [training_df, test_df]

In [3]:
for df in dataframes:
    df.columns.values[0]="Id"
    df[['MonthlyIncome','NumberOfDependents']] = df[['MonthlyIncome','NumberOfDependents']].fillna(0)
    df['NumberOfTimes30DaysOrMoreLate'] = df['NumberOfTime30-59DaysPastDueNotWorse']+df['NumberOfTime60-89DaysPastDueNotWorse']+df['NumberOfTimes90DaysLate']
    df['IncomePerPerson'] = df['MonthlyIncome']/(df['NumberOfDependents']+1)
    df['MonthlyDebt'] = df['MonthlyIncome']*df['DebtRatio']
    df['MonthlyBalance'] = df['MonthlyIncome']-df['MonthlyDebt']

In [4]:
training_df.to_csv('./results/full_train_modif.csv', index=False)
test_df.to_csv('./results/test_modif.csv', index=False)

In [5]:
full_train_source = api.create_source('./results/full_train_modif.csv')
api.ok(full_train_source)

True

In [6]:
full_train_dataset = api.create_dataset(full_train_source)
api.ok(full_train_dataset)

True

In [7]:
dataset_train80 = api.create_dataset(full_train_dataset,{"name":"Train", "sample_rate":0.8, "seed":"my seed"})
api.ok(dataset_train80)

True

In [8]:
dataset_valid20 = api.create_dataset(full_train_dataset, {"name":"Validation", "sample_rate":0.8, "seed":"my seed", "out_of_bag":True})
api.ok(dataset_valid20)

True

In [9]:
ensemble = api.create_ensemble(dataset_train80, {"objective_field":"SeriousDlqin2yrs"})
api.ok(ensemble)

True

In [10]:
test_source = api.create_source('./results/test_modif.csv')
api.ok(test_source)

True

In [13]:
test_dataset = api.create_dataset(test_source)
api.ok(test_dataset)

True

In [14]:
batch_prediction = api.create_batch_prediction(ensemble, test_dataset, {'output_fields':['Id'], 'probabilities':True})
api.ok(batch_prediction)

True

In [15]:
api.download_batch_prediction(batch_prediction, filename= './results/my_predictions.csv')

'./results/my_predictions.csv'

In [16]:
evaluation = api.create_evaluation(ensemble, dataset_valid20)
api.ok(evaluation)

True

In [17]:
my_prediction_df = pd.read_csv('./results/my_predictions.csv', index_col = 0)
my_prediction_df = my_prediction_df.drop(columns=['SeriousDlqin2yrs','0 probability'])
my_prediction_df = my_prediction_df.rename(columns={"1 probability": "Probability"})
my_prediction_df

,Probability
Id,
1,0.05564
2,0.04129
3,0.01519
4,0.06939
5,0.12526
...,...
101499,0.02252
101500,0.35555
101501,0.00274


In [18]:
my_prediction_df.to_csv('./results/prediction-kaggle.csv')

In [23]:
matrice_confusion = evaluation['object']['result']['model']['confusion_matrix']
pprint(matrice_confusion)

[[27733, 252], [1689, 326]]


In [25]:
roc = evaluation['object']['result']['model']['average_area_under_roc_curve']
pprint(roc)

0.85441


In [26]:
import kaggle
kaggle.api.competition_submit("./results/prediction-kaggle.csv", "BigML ensemble", "GiveMeSomeCredit")

100%|██████████| 1.65M/1.65M [00:05<00:00, 332kB/s] 


Successfully submitted to Give Me Some Credit